In [3]:
import os
import pickle
import napari
import nd2
import numpy as np
from skimage.draw import polygon
from skimage.io import imsave

In [4]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'
im_name = r'092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2'
path_im = os.path.join(path_dir,im_name)

path_save = r'D:\data_analysis\2022_Sahana\masks'

In [5]:
im = nd2.imread(path_im)

In [6]:
im.shape

(27, 4, 512, 512)

In [7]:
# create a maximum projection

actin_max = np.max(im[:,0,:,:],axis=0)

In [33]:
viewer = napari.Viewer()
viewer.add_image(im[:,0,:,:],colormap='magenta',blending='additive')
viewer.add_image(im[:,1,:,:],colormap='red',blending='additive')
viewer.add_image(im[:,2,:,:],colormap='green',blending='additive')
viewer.add_image(im[:,3,:,:],colormap='blue',blending='additive')
viewer.add_image(actin_max,blending='additive')

<Image layer 'actin_max' at 0x2aa309d7c10>

In [34]:
# Manually define cells 
viewer.add_shapes()

c:\Users\lab\.conda\envs\napari-env\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Shapes layer 'Shapes' at 0x2aa34966040>

## Manually draw shapes

Use the set of tools available in Napari to draw the contours of the cells. Do not include cells that are not complete.

## Save shapes

In [35]:
# get coordinates of vertices
vertices_polygons = viewer.layers['Shapes'].data
len(vertices_polygons)

6

In [36]:
# create a mask out of vertices
mask = np.zeros(im.shape[2:]).astype('uint8')

for i,poly in enumerate(vertices_polygons):

    # if drawing was in 3D
    if len(poly.shape) > 2:
        mask_coord = polygon(vertices_polygons[i][:,1],vertices_polygons[i][:,2],shape=im.shape[2:])
    else:
        mask_coord = polygon(vertices_polygons[i][:,0],vertices_polygons[i][:,1],shape=im.shape[2:])

    mask[mask_coord] = i+1

In [37]:
# test identification of a mask
viewer.add_labels(mask)

<Labels layer 'mask' at 0x2aa2f142130>

In [38]:
# saving
mask_path = os.path.join(path_save,im_name.replace('.nd2','_mask.png'))
imsave(mask_path,mask)


pkl_path = os.path.join(path_save,im_name.replace('.nd2','_polygons.pkl'))
with open(pkl_path, 'wb') as f:
    pickle.dump(vertices_polygons, f)

C:\Users\lab\AppData\Local\Temp\ipykernel_23524\2345319060.py:3: UserWarning: D:\data_analysis\2022_Sahana\masks\092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001_mask.png is a low contrast image
  imsave(mask_path,mask)
